In [ ]:
from google.colab import auth
from bs4 import BeautifulSoup
import pandas as pd
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
print('WELCOME TO INSTAPAAAR!')
html_content = input('Enter the HTML from the Instacart Receipt : ')

#Pantars
pantars = {'Pritish': 'pthombare@cs.stonybrook.edu', 'Smit':'skumbhani@cs.stonybrook.edu', 'Mitesh':'msjalan@cs.stonybrook.edu', 'Mohammad':'mkhakhariawa@cs.stonybrook.edu', 'Anim':'animesh.uttekar@stonybrook.edu'}

soup = BeautifulSoup(html_content, 'html.parser')
title_div = soup.find('div', class_='DriverDeliverySchedule')
order_title = title_div.text.strip()
print("Order title : ", order_title)

try :
   items_table = soup.find('table', class_='items delivered')
   if items_table == None:
       raise Exception("Items table not found")
except Exception as e:
    print('Error in parsing the html content : ', e)

items_headers = ['Item name', 'Item price']
items_rows = []

for row in items_table.find_all('tr'):
    item_cell = row.find_all('td', class_='order-item')
    if len(item_cell) != 0:
        div_item_name = item_cell[0].find('div', class_='item-name')
        names = div_item_name.text.split('\n\n')
        names = [name.strip() for name in names if name != '\n']
        product_name = ' | '.join(names)
        div_item_price = item_cell[0].find('div', class_='total')
        product_price = div_item_price.text.strip()
        row_data = [product_name, product_price]
        items_rows.append(row_data)

items_df = pd.DataFrame(items_rows, columns=items_headers)
for pantar in pantars.keys():
  items_df[pantar] = 0
print(items_df)

def removeDollar(amount):
  dollar_removed = amount.replace('$', '')
  return float(dollar_removed)

#Charges

try :
   charges_table = soup.find('table', class_='charges')
   if charges_table == None:
       raise Exception("Charges table not found")
except Exception as e:
    print('Error in parsing the html content : ', e)

charges_headers = ['Charge type', 'Charge']
charges_rows = []
common_charges = 0
final_amount = 0
for row in charges_table.find_all('tr'):
  td_charge = row.find('td', class_='charge-type')
  charge_type = ''
  if td_charge != None:
    charge_type = td_charge.text.strip()
  td_amount = row.find('td', class_='amount')
  amount = ''
  if td_amount != None:
    amount = td_amount.text.strip()
  if charge_type not in ['Items Subtotal', 'Instacart+ Member Free Delivery!', 'You saved', 'Total']:
    common_charges += removeDollar(amount)
  if charge_type == 'Total':
    final_amount = amount
  row_data = [charge_type, amount]
  charges_rows.append(row_data)

charges_df = pd.DataFrame(charges_rows, columns=charges_headers)
print(charges_df)
print('Common charges : ', common_charges)
print('Total debited amount : ', final_amount)

main_sheet = gc.create(order_title)
spreadsheet = gc.open(order_title)
items_worksheet = spreadsheet.get_worksheet(0)
items_worksheet.update("A1", [items_df.columns.values.tolist()] + items_df.values.tolist())
charges_worksheet = spreadsheet.add_worksheet(title="Charges", rows=100, cols=100)
charges_worksheet.update('A1', 'Common charges')
charges_worksheet.update('B1', str(common_charges))
charges_worksheet.update('A2', 'Common charges per person')
charges_worksheet.update('B2', str(common_charges/len(pantars)))
charges_worksheet.update("A5", [charges_df.columns.values.tolist()] + charges_df.values.tolist())
email_list = pantars.values()

for email in email_list:
    main_sheet.share(email, perm_type='user', role='writer')

print("Google Sheet shared successfully!")


WELCOME TO INSTAPAAAR!
Enter the HTML from the Instacart Receipt :  <!DOCTYPE html> <html lang='en'> <!--                                           `-:///:-.                                               /ssssssssso/.                                              :sssssssssssss/                                              +ssssssssssssss`                                              +ssssssssssssso                                              :sssssssssssso`                                              `ossssssssss/`                                               :ssssssss+.  `.-.`                                                /sssss+. `:osssso/                                     .:///::-`   :+/-` `/ossssssss/                                  `-/+++++++++/:.     :osssssssssss`                                 -/++++++++++++++/.  `ossssssssssso                               `/++++++++++++++++++/`  .-/++ooso+/`                              -++++++++++++++++++++++-                        

Google Sheet shared successfully!
